In [2]:
import geopandas as gpd
import pandas as pd
from sqlalchemy import create_engine
from haversine import haversine
from tqdm.notebook import tqdm
import numpy as np

In [45]:
from sqlalchemy import create_engine
import pymysql

endpoint = 'dao.cdqd2bwz9q4s.ap-northeast-2.rds.amazonaws.com'
db_connection_str = 'mysql+pymysql://admin:ekfkawnl@{}/{}'.format(endpoint, 'dao')
try :
    db_connection = create_engine(db_connection_str)
    conn = db_connection.connect()
except :
    print('fail to connect db')

# 생태통로 목록 전처리

In [27]:
df = pd.read_csv('Mapservice2022-08-08.csv', encoding='cp949')

In [28]:
col = list(df.columns)
col

['번호',
 '생태통로명',
 '주소',
 '도로명',
 '도로번호',
 '차선_수',
 '위도_도',
 '위도_분',
 '위도_초',
 '경도_도',
 '경도_분',
 '경도_초']

In [29]:
for c in col[-6:]:
    try :
        if '_도' in c :
            df[c] = df[c].map(int)
        else :
            df[c] = df[c].map(float)
    except Exception as e:
        print(e)
        print(c)

invalid literal for int() with base 10: '36+'
위도_도
could not convert string to float: '208"'
위도_초
could not convert string to float: '413"'
경도_초


In [36]:
print(df[df['위도_도'] == '36+'].index)
print(df[df['위도_초']=='208"'].index)
print(df[df['경도_초']=='413"'].index)

Int64Index([454], dtype='int64')
Int64Index([26], dtype='int64')
Int64Index([26], dtype='int64')


In [37]:
df.loc[454, '위도_도'] = 36
df.loc[26, '위도_초'] = 208
df.loc[26, '경도_초'] = 413

In [38]:
for c in col[-6:]:
    try :
        if '_도' in c :
            df[c] = df[c].map(int)
        else :
            df[c] = df[c].map(float)
    except Exception as e:
        print(e)
        print(c)

### 도분초 열 만들기

In [39]:
lat = list(zip(df['위도_도'].astype('int32'), df['위도_분'].astype('int32'), df['위도_초']))
lon = list(zip(df['경도_도'].astype('int32'), df['경도_분'].astype('int32'), df['경도_초']))

new_lat = [' '.join(map(str, l)) for l in lat]

new_lon = [' '.join(map(str, l)) for l in lon]

df['위도_도분초'] = new_lat
df['경도_도분초'] = new_lon

### 십진법 열 만들기

In [40]:
def change_GPS(df):
  Latitude_D=df['위도_도']
  Latitude_M=df['위도_분']
  Latitude_S=df['위도_초']
  Longitude_D=df['경도_도']
  Longitude_M=df['경도_분']
  Longitude_S=df['경도_초']

  Latitude=Latitude_D+(Latitude_M/60)+(Latitude_S/3600)
  Longitude=Longitude_D+(Longitude_M/60)+(Longitude_S/3600)

  Latitude = round(Latitude, 5)
  Longitude = round(Longitude, 5)

  df['위도_십진법'] = Latitude
  df['경도_십진법'] = Longitude
  return True

In [41]:
change_GPS(df)

True

In [42]:
df

,번호,생태통로명,주소,도로명,도로번호,차선_수,위도_도,위도_분,위도_초,경도_도,경도_분,경도_초,위도_도분초,경도_도분초,위도_십진법,경도_십진법
0,1001,중동터널,서울특별시 구로구 궁동,계남큰길,NaN,왕복6차로,37,30.0,23.37000,126,49.0,51.56000,37 30 23.37,126 49 51.56,37.50649,126.83099
1,1002,개좌생태터널,부산광역시 금정구 회동동 산4-1,개좌로,0,왕복2차선,35,15.0,6.05000,129,8.0,27.31000,35 15 6.05,129 8 27.31,35.25168,129.14092
2,1004,곰내 생태통로,부산광역시 기장군 철마면 웅천리,군도,19호선,왕복2차선,35,18.0,20.80000,129,10.0,19.20000,35 18 20.8,129 10 19.2,35.30578,129.17200
3,1005,정관예림리 생태통로,부산광역시 기장군 정관면 예림리,국지도,60호,왕복4차선,35,19.0,46.80000,129,12.0,11.90000,35 19 46.8,129 12 11.9,35.32967,129.20331
4,1006,을숙도 생태통로,부산광역시 사하구 하단동,낙동남로,1240,왕복7차선,35,6.0,31.84632,128,56.0,28.46904,35 6 31.8463200000076,128 56 28.469039999984,35.10885,128.94124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,1564,삼척노곡,강원도 삼척시 원덕읍 노곡리 산146-3,국도,7호선,왕복4차선,37,12.0,33.19000,129,20.0,14.09000,37 12 33.19,129 20 14.09,37.20922,129.33725
532,1565,절토육교,경기도 의왕시 월암동 산46-3,서수원~의왕간 도로,지방도 309호선,왕복6차선,37,19.0,2.04000,126,57.0,59.00000,37 19 2.04,126 57 59.0,37.31723,126.96639
533,1566,동물이동통로,경기도 의왕시 고천동 산55-5,서수원~의왕간 도로,지방도309호선,왕복6차선,37,20.0,11.80000,126,58.0,23.42000,37 20 11.8,126 58 23.42,37.33661,126.97317
534,1567,NaN,경기도 영중면 거사리 산3-22번지 일원,중로,3-30,왕복2차선,37,59.0,30.60000,127,13.0,43.23000,37 59 30.6,127 13 43.23,37.99183,127.22868


In [43]:
df.to_csv('생태통로_목록_0808.csv', encoding='cp949', index=False)

In [46]:
df.to_sql('생태통로_목록_0808', db_connection, index=False)

536

In [47]:
df['번호'].nunique()

536